In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.linear_model import LassoCV
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV


In [2]:
df = pd.read_csv(
    '2025-10-14-28.csv',
    sep=';',
    encoding='utf-8',
    encoding_errors='replace',  # replaces bad chars with �
    engine='python'
)


In [3]:
location_df = pd.read_excel('hotel_and_location2.xlsx')
df = df.merge(location_df, on="Name", how="left")
df = df.dropna(subset=['Competitor Link'])
df['distance'].describe()
col = 'Competitor Price'

Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame
df_filtered = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

df['Checkin'] = pd.to_datetime(df['Checkin'])
df['Scraping Date'] = pd.to_datetime(df['Scraping Date'])

df['DayName'] = df['Checkin'].dt.day_name()
df['Competitor Price'] = df['Competitor Price'].astype(float)
df.loc[df['DayName'].isin(['Friday', 'Saturday']), 'Competitor Price'] /= 2

df['LogPrice'] = np.log1p(df['Competitor Price'])
df['LeadTime'] = (df['Checkin'] - df['Scraping Date']).dt.days

df['Checkout'] = pd.to_datetime(df['Checkout'])
df['LengthOfStay'] = (df['Checkout'] - df['Checkin']).dt.days


In [4]:
cz_holidays = holidays.Czechia(years=range(df['Checkin'].dt.year.min(), 
                                           df['Checkin'].dt.year.max() + 1))

df['Holiday'] = df['Checkin'].apply(lambda x: 1 if x in cz_holidays else 0)

df['IsWeekend'] = df['DayName'].isin(['Friday', 'Saturday']).astype(int)

df['Month'] = df['Checkin'].dt.month

df["My Room"].unique()

my_hotel = 'karlova-prague'
df_my = df[df['Name'] == my_hotel].copy()
df_comp = df[df['Name'] != my_hotel].copy()

# Optional: reset index
df_my.reset_index(drop=True, inplace=True)
df_comp.reset_index(drop=True, inplace=True)

In [5]:
comp_stats = (
    df_comp.groupby(['My Room', 'Checkin', 'Scraping Date'])['LogPrice']
           .agg(['mean','min','max'])
           .reset_index()
           .rename(columns={
               'mean': 'Competitor Log Avg Price',
               'min': 'Competitor Log Min Price',
               'max': 'Competitor Log Max Price'
           })
)

# 4. Merge stats back to your hotel rows
df = df.merge(
    comp_stats,
    on=['My Room', 'Checkin', 'Scraping Date'],
    how='left'
)

In [6]:
df = pd.get_dummies(df, columns=['My Room', 'DayName'])

features = [
    'Holiday',
    'Competitor Breakfast',
    'Competitor Nonref',
    'Month',
    'IsWeekend',
    'Competitor Log Avg Price',
    'Competitor Log Min Price',
    'Competitor Log Max Price',
    'LeadTime',
    'distance',
    'square'
] + \
[col for col in df.columns if col.startswith('My Room_')] + \
[col for col in df.columns if col.startswith('DayName_')]




In [7]:
numeric_cols = ['LeadTime', 'Competitor Log Avg Price', 'Competitor Log Min Price',
                'Competitor Log Max Price','distance']
scaler = StandardScaler()


In [8]:
target = 'LogPrice'

X = df[features]
y = df[target]

model = RandomForestRegressor(n_estimators=100, random_state=42)
# --- Train-test split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

In [9]:
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42) 
# --- Train XGBoost model --- 
model = XGBRegressor() 
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False) 
# --- Predict and evaluate --- 
y_pred = model.predict(X_test) 
mae = mean_absolute_error(y_test, y_pred) 
print(f"MAE for {my_hotel}: {mae:.2f}") 
mae_price = mean_absolute_error(np.exp(y_test), np.exp(y_pred)) 
print("MAE (actual price):", mae_price)

MAE for karlova-prague: 0.13
MAE (actual price): 32.25163134134363


In [11]:
importance_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Importance": model.feature_importances_
}).sort_values("Importance", ascending=False)

print(importance_df)


                                              Feature  Importance
5                            Competitor Log Avg Price    0.250261
17  My Room_Rodinný apartmán se 2 ložnicemi a výhl...    0.240164
13  My Room_Dvoulůžkový pokoj Deluxe s manželskou ...    0.064555
12      My Room_Double room with Bath and Kitchenette    0.052242
11                   My Room_Attic Suite with Kitchen    0.049675
2                                   Competitor Nonref    0.039993
10                                             square    0.037229
15     My Room_Dvoulůžkový pokoj s manželskou postelí    0.037166
9                                            distance    0.036135
20                 My Room_Čtyřlůžkový pokoj Superior    0.034936
1                                Competitor Breakfast    0.033448
14  My Room_Dvoulůžkový pokoj Standard s manželsko...    0.020105
18                     My Room_Rodinný pokoj Superior    0.017572
19                  My Room_Třílůžkový pokoj Superior    0.015900
7         